In [1]:
import numpy as np
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(42)
random.seed(42)

from DataExploration import DataExploration
from DecisionTree import DecisionTree
from NeuralNetwork import DNN
from Boosting import Boosting
from KNN import KNN
from NeuralNetworkKeras import NN_Keras
from SVM import SVM

C:\Users\matan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


C:\Users\matan\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.


In [2]:
# get datasets ready
de = DataExploration()

twitter_df = de.prepare_twitter_df()
speed_dating_df = de.prepare_speed_dating_df()
encoded_speed_dating_df, labelled_speed_dating_df, label_encoder = de.get_hot_encoded_speed_dating(speed_dating_df)

(22223, 10)


(19276, 10)
(41499, 10)


(8378, 44)
removing empty values shape:
(6794, 44)


In [3]:
X_train, X_test, y_train, y_test = de.get_train_test_validation(encoded_speed_dating_df, 'speed_dating')

In [4]:
# de.investigate_twitter(twitter_df)
# de.investigate_speed_dating(labelled_speed_dating_df, label_encoder)

['num_following', 'num_followers', 'num_tweets', 'len_of_screen_name', 'len_of_profile_description', 'num_following_changes']


C:\Users\matan\Anaconda3\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


['num_following', 'num_followers', 'num_tweets', 'len_of_screen_name', 'len_of_profile_description', 'num_following_changes']


(6794, 44)
(array([3336.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
       3458.]), array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]), <a list of 10 Patch objects>)
(array([1675.,    0.,  340.,    0.,    0.,  503.,    0., 3824.,    0.,
        452.]), array([0. , 0.4, 0.8, 1.2, 1.6, 2. , 2.4, 2.8, 3.2, 3.6, 4. ]), <a list of 10 Patch objects>)
(array([1641.,    0.,  355.,    0.,    0.,  556.,    0., 3795.,    0.,
        447.]), array([0. , 0.4, 0.8, 1.2, 1.6, 2. , 2.4, 2.8, 3.2, 3.6, 4. ]), <a list of 10 Patch objects>)
(array([ 434.,  404.,  444.,  590.,  471.,  692.,  336., 1277., 1416.,
        730.]), array([  0. ,  24.7,  49.4,  74.1,  98.8, 123.5, 148.2, 172.9, 197.6,
       222.3, 247. ]), <a list of 10 Patch objects>)


C:\Users\matan\OneDrive\Documents\OMSCS\CS7641\assignments\code\CS7641-assignment1\DataExploration.py:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data_speed_dating['label'] = np.where(data_speed_dating['decision'] == 1, 'positive_date', 'negative_date')


['importance_same_race', 'importance_same_religion', 'field', 'attractive_important', 'sincere_important', 'intellicence_important', 'funny_important', 'ambtition_important', 'shared_interests_important', 'attractive', 'sincere', 'intelligence', 'funny', 'ambition', 'attractive_partner', 'sincere_partner', 'intelligence_partner', 'funny_partner', 'ambition_partner', 'shared_interests_partner', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'interests_correlate']




# Speed Dating Learners


In [4]:
#best_params = {'dt__alpha': -0.01, 'dt__max_depth': 9, 'dt__min_samples_leaf': 7} #twitter
#best_params = {'dt__alpha': -0.1, 'dt__max_depth': 9, 'dt__min_samples_leaf': 2} #speed_dating  (CV score=0.724):
dt_clf = DecisionTree(model_name='dt_pruning_1', dataset_name='speed_dating')
dt_clf.train(X_train, y_train)
y_pred = dt_clf.predict(X_test)
dt_clf.evaluate_model(y_test=y_test, y_pred=y_pred)

#print(dt_clfr.get_model().best_estimator_.named_steps['dt'].tree_.max_depth)


In [ ]:
dt_best_params = dt_clf.get_best_params()
print(dt_best_params)
dt_clf_model = dt_clf.get_model()
#print(dt_clf.feature_importance_)


None


In [ ]:
#best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}
#best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 40} (CV score=0.772):
boosting_clf = Boosting(model_name='boosting_1', dataset_name='speed_dating')
boosting_clf.train(X_train, y_train)
y_pred = boosting_clf.predict(X_test)
boosting_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


ValueError: Number of weights=3505 does not match number of samples=2804

In [ ]:
boosting_best_params = boosting_clf.get_best_params()
print(boosting_best_params)
boosting_clf_model = boosting_clf.get_model()

None


In [9]:
#best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'} #twitter
#{'knn__metric': 'manhattan', 'knn__n_neighbors': 121, 'knn__weights': 'distance'} (CV score=0.749): #speeddating
knn_clf = KNN(model_name='knn_1', dataset_name='speed_dating')
knn_clf.train(X_train, y_train)
y_pred = knn_clf.predict(X_test)
knn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


Best parameter (CV score=0.749):
{'knn__metric': 'manhattan', 'knn__n_neighbors': 121, 'knn__weights': 'distance'}


[[562  72]
 [189 273]]
             precision    recall  f1-score   support

        0.0       0.75      0.89      0.81       634
        1.0       0.79      0.59      0.68       462

avg / total       0.77      0.76      0.75      1096



In [10]:
knn_best_params = knn_clf.get_best_params()
print(knn_best_params)
knn_clf_model = knn_clf.get_model()

{'knn__metric': 'manhattan', 'knn__n_neighbors': 121, 'knn__weights': 'distance'}


In [11]:
#best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'} #twitter
#{'knn__metric': 'manhattan', 'knn__n_neighbors': 49, 'knn__weights': 'distance'} (CV score=0.728): #speeddating
knn_clf = KNN(model_name='knn_1', dataset_name='speed_dating')
knn_clf.train(X_train, y_train)
y_pred = knn_clf.predict(X_test)
knn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


Best parameter (CV score=0.749):
{'knn__metric': 'manhattan', 'knn__n_neighbors': 121, 'knn__weights': 'distance'}


[[562  72]
 [189 273]]
             precision    recall  f1-score   support

        0.0       0.75      0.89      0.81       634
        1.0       0.79      0.59      0.68       462

avg / total       0.77      0.76      0.75      1096



In [12]:
#best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'} #twitter
#{'knn__metric': 'manhattan', 'knn__n_neighbors': 49, 'knn__weights': 'distance'} (CV score=0.728): #speeddating
knn_clf = KNN(model_name='knn_1', dataset_name='speed_dating')
knn_clf.train(X_train, y_train)
y_pred = knn_clf.predict(X_test)
knn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


KeyboardInterrupt: 

In [13]:
#best_params = {'batch_size': 10, 'epochs': 1, 'layer_sizes': [12, 1], 'learning_rate': 0.0005,
#               'loss': 'mean_squared_error'}
#best_params = {'dnn__batch_size': 100, 'dnn__epochs': 50, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'mean_squared_error'} #
best_params = {'dnn__batch_size': 50, 'dnn__epochs': 30, 'dnn__layer_sizes': [100, 20], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'binary_crossentropy'} #
keras_nn_clf = NN_Keras(model_name='keras_nn_1', dataset_name='speed_dating', best_params=best_params)
keras_nn_clf.train(X_train, y_train)
y_pred = keras_nn_clf.predict(X_test)
keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)



Epoch 1/30


  50/4382 [..............................] - ETA: 1:00 - loss: 8.9277 - acc: 0.4400

 900/4382 [=====>........................] - ETA: 2s - loss: 9.5831 - acc: 0.3989  

1950/4382 [============>.................] - ETA: 0s - loss: 9.1321 - acc: 0.4272

3100/4382 [====================>.........] - ETA: 0s - loss: 9.0717 - acc: 0.4310

...

3550/4382 [=======================>......] - ETA: 0s - loss: 9.0355 - acc: 0.4332

4382/4382 [==============================] - 0s 62us/step - loss: 9.0335 - acc: 0.4334


Epoch 26/30
  50/4382 [..............................] - ETA: 0s - loss: 8.9277 - acc: 0.4400

 800/4382 [====>.........................] - ETA: 0s - loss: 9.2665 - acc: 0.4188

KeyboardInterrupt: 

In [14]:
keras_nn_best_params = keras_nn_clf.get_best_params()
print(keras_nn_best_params)

keras_nn_clf_model = keras_nn_clf.get_model()

{'dnn__batch_size': 50, 'dnn__epochs': 30, 'dnn__layer_sizes': [100, 20], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'binary_crossentropy'}


In [15]:
#best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
best_params = {'svm__C': 16.0, 'svm__gamma': 0.00390625, 'svm__kernel': 'rbf'} # (CV score=0.781)
svm_clf_rbf = SVM(model_name='svm_rbf_1', dataset_name='speed_dating', best_params=best_params)
svm_clf_rbf.train(X_train, y_train,'rbf')
y_pred = svm_clf_rbf.predict(X_test)
svm_clf_rbf.evaluate_model(y_test=y_test, y_pred=y_pred)


KeyboardInterrupt: 

In [16]:
svm_best_params = svm_clf_rbf.get_best_params()
print(svm_best_params) 
svm_clf_rbf_model = svm_clf_rbf.get_model()
#{'svm__degree': 1, 'svm__kernel': 'rbf'}
# (CV score=0.781):
# {'svm__C': 16.0, 'svm__gamma': 0.00390625, 'svm__kernel': 'rbf'}

{'svm__C': 16.0, 'svm__gamma': 0.00390625, 'svm__kernel': 'rbf'}


In [17]:
#best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
best_params = {'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} #r (CV score=0.764):

svm_clf_poly = SVM(model_name='svm_poly_1', dataset_name='speed_dating', best_params=best_params)
svm_clf_poly.train(X_train, y_train,'poly')
y_pred = svm_clf_poly.predict(X_test)
svm_clf_poly.evaluate_model(y_test=y_test, y_pred=y_pred)


[[498 136]
 [131 331]]
             precision    recall  f1-score   support

        0.0       0.79      0.79      0.79       634
        1.0       0.71      0.72      0.71       462

avg / total       0.76      0.76      0.76      1096



In [18]:
svm_best_params = svm_clf_poly.get_best_params()
print(svm_best_params) 
svm_clf_poly_model = svm_clf_poly.get_model()
#{'svm__degree': 1, 'svm__kernel': 'rbf'}

{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'}


In [19]:
# #best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
# #{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} r (CV score=0.764):
# 
# svm_clf_linear_sigmoid = SVM(model_name='svm_linear_sigmoid_1', dataset_name='speed_dating')
# svm_clf_linear_sigmoid.train(X_train, y_train,'linear_sigmoid')
# y_pred = svm_clf_linear_sigmoid.predict(X_test)
# svm_clf_linear_sigmoid.evaluate_model(y_test=y_test, y_pred=y_pred)


In [20]:
# svm_best_params = svm_clf_linear_sigmoid.get_best_params()
# print(svm_best_params) 
# svm_clf_linear_sigmoid_model = svm_clf_linear_sigmoid.get_model()
# #{'svm__degree': 1, 'svm__kernel': 'rbf'}

# Twitter Learners

In [21]:
X_train, X_test, y_train, y_test, = de.get_train_test_validation(twitter_df, 'twitter')


ValueError: not enough values to unpack (expected 8, got 4)

In [22]:
#best_params = {'dt__alpha': -0.01, 'dt__max_depth': 9, 'dt__min_samples_leaf': 7} #twitter
#best_params = {'dt__alpha': -0.1, 'dt__max_depth': 9, 'dt__min_samples_leaf': 2} #speed_dating  (CV score=0.724):
dt_clf_t = DecisionTree(model_name='dt_pruning_1', dataset_name='twitter')
dt_clf_t.train(X_train, y_train)
y_pred = dt_clf_t.predict(X_test)
dt_clf_t.evaluate_model(y_test=y_test, y_pred=y_pred)

#print(dt_clfr.get_model().best_estimator_.named_steps['dt'].tree_.max_depth)


ValueError: Number of weights=3505 does not match number of samples=2804

In [23]:
dt_best_params = dt_clf_t.get_best_params()
print(dt_best_params)
dt_clf_t_model = dt_clf_t.get_model()
#print(dt_clf.feature_importance_)


None


In [24]:
#best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}
#best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 40} (CV score=0.772):
boosting_clf_t = Boosting(model_name='boosting_1', dataset_name='twitter')
boosting_clf_t.train(X_train, y_train)
y_pred = boosting_clf_t.predict(X_test)
boosting_clf_t.evaluate_model(y_test=y_test, y_pred=y_pred)


ValueError: Number of weights=3505 does not match number of samples=2804

In [25]:
boosting_best_params = boosting_clf_t.get_best_params()
print(boosting_best_params)
boosting_clf_t_model = boosting_clf.get_model()

None


In [26]:
#best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'} #twitter
#{'knn__metric': 'manhattan', 'knn__n_neighbors': 49, 'knn__weights': 'distance'} (CV score=0.728): #speeddating
knn_clf_t = KNN(model_name='knn_1', dataset_name='twitter')
knn_clf_t.train(X_train, y_train)
y_pred = knn_clf_t.predict(X_test)
knn_clf_t.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
best_params = {'svm__C': 16.0, 'svm__gamma': 1.0, 'svm__kernel': 'rbf'}
svm_clf_rbf_t = SVM(model_name='svm_rbf_1', dataset_name='twitter', best_params=best_params)
svm_clf_rbf_t.train(X_train, y_train,'rbf')
y_pred = svm_clf_rbf_t.predict(X_test)
svm_clf_rbf_t.evaluate_model(y_test=y_test, y_pred=y_pred)

In [ ]:
svm_best_params = svm_clf_rbf_t.get_best_params()
print(svm_best_params) 
svm_clf_rbf_t_model = svm_clf_rbf_t.get_model()
#Best parameter (CV score=0.893):
#{'svm__C': 16.0, 'svm__gamma': 1.0, 'svm__kernel': 'rbf'}

In [ ]:
# #best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
# #{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} r (CV score=0.764):
# 
# svm_clf_poly = SVM(model_name='svm_wip_poly', dataset_name='twitter')
# svm_clf_poly.train(X_train, y_train,'poly')
# y_pred = svm_clf_poly.predict(X_test)
# svm_clf_poly.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
# svm_best_params = svm_clf_poly.get_best_params()
# print(svm_best_params) 
# #{'svm__degree': 1, 'svm__kernel': 'rbf'}

In [ ]:
# #best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
# #{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} r (CV score=0.764):
# 
# svm_clf_linear_sigmoid = SVM(model_name='svm_wip_linear_sigmoid', dataset_name='twitter')
# svm_clf_linear_sigmoid.train(X_train, y_train,'linear_sigmoid')
# y_pred = svm_clf_linear_sigmoid.predict(X_test)
# svm_clf_linear_sigmoid.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
# svm_best_params = svm_clf_linear_sigmoid.get_best_params()
# print(svm_best_params) 
# #{'svm__degree': 1, 'svm__kernel': 'rbf'}

In [ ]:
#best_params = {'batch_size': 10, 'epochs': 1, 'layer_sizes': [12, 1], 'learning_rate': 0.0005,
#               'loss': 'mean_squared_error'}
#{'dnn__batch_size': 10, 'dnn__epochs': 1, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'mean_squared_error'} #
keras_nn_clf = NN_Keras(model_name='keras_nn_wip1', dataset_name='spam_twitter')
keras_nn_clf.train(X_train, y_train)
y_pred = keras_nn_clf.predict(X_test)
keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
keras_nn_best_params = keras_nn_clf.get_best_params()
print(keras_nn_best_params)